# **Introdução**

O objetivo desse trabalho é implementar "Automated Bias Discover Task" descrito em: _Discover the Unknown Biased Attribute of an Image Classifier
Zhiheng Li, Chenliang Xu_

O workflow descrito no paper, pode ser bem resumido pela seguinte imagem:

# ![title](images/workflow.png)

Percaba que primeiramente precisamos obter um hiperplano do atributo enviesado e, depois disso, a tarefa de encontrar o viés do classificador continua.
Nesse sentido, o escopo desse trabalho é, primeiramente obter esse hiperplano otimizado.

Para fazer isso, precisamos de duas coisas:
1. Um classificador enviesado ✅
2. Um modelo gerativo capaz de reproduzir a distribuição de probabilidades do dataset desse classificador ✅

Como já possuímos esses dois pre-requisitos, podemos implementar esse workflow. É importante perceber que faremos uma abordagem 1 vs all para cada classe no problema. Podendo encontrar diversos vieses presentes nesse dataset, um para cada classe.

# **Implementação**

Para implementar esse precisamos saber a dimensão $d$ do espaço latente do modelo gerativo treinado. Essa será a dimensionalidade do "hiperplano do atributo enviesado" que será encontrado.

In [2]:
import torch
from torch import nn

In [ ]:
class BiasDicoverer(nn.Module):
    def __init__(self, z_dim, generative_model, classifier):
        self.hyperplane = nn.Parameter(torch.rand(z_dim))
        self.generative_model = generative_model
        self.classifier = classifier
    
    def generate_latent_codes(hyperplane):

    
    def forward():
        latent_codes = generate_latent_codes(hyperplane)
        traversal_images = self.generativel_model(latent_codes)
        labels = self.classifier(traversal_images)

In [ ]:
def TotalVARLoss(num_traversal_images, predictions):
    

In [ ]:
def optimize_hyperplane():
    